# Hybrid approach

Idea to limit output token field of abstractive summarizer with input text lexicon.

In [7]:
!pip install transformers sentencepiece

In [8]:
!pip install spacy

In [11]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 27.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
model = T5ForConditionalGeneration.from_pretrained("t5-large")
tokenizer = T5Tokenizer.from_pretrained("t5-large")

In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [12]:
body = "Resonance is the literary magazine put out by the students of Falmouth Academy, the Massachusetts private school I attended for six years, starting in the seventh grade. During my time at F.A., I had at least one poem published in each issue of Resonance. In high school, I was also a member of the staff. But that wasn’t why I loved it. I loved it — and I swear I am not exaggerating here — because I thought the writing in its pages was more beautiful than anything I’d ever read. I was not a happy or popular adolescent, and the emotional stance I adopted toward most of my peers at F.A. might best be described as a defensive crouch. I was scared of my classmates, and I resented them; I could tell they didn’t like me, but I couldn’t figure out why. To the extent that I was able to lift myself out of my own sodden self-loathing to contemplate their inner worlds, I imagined their minds to be filled, like mine, with a whirlwind of criticism and judgment. But, once a year, at the end of the spring semester, I would open my copy of Resonance and be forced to face the unsettling possibility that my classmates were not the shallow bullies I imagined them to be but actual people, with souls."

In [14]:
doc = nlp(body)
sentences = [sent for sent in doc.sents]

In [18]:
sentences[1].text

'During my time at F.A., I had at least one poem published in each issue of Resonance.'

In [25]:
summarized_sentences = []

In [28]:
for sentence in sentences:
  text = "summarize :" + sentence.text
  tokens_input = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids = model.generate(tokens_input,
                            # min_length=60,
                            max_length=tokens_input.shape[1]
  )
  summary = tokenizer.decode(summary_ids[0])
  summarized_sentences.append(summary)
  print(">> Original text:")
  print(sentence)
  print(">> Summary:")
  print(summary)
  print("------------")

>> Original text:
Resonance is the literary magazine put out by the students of Falmouth Academy, the Massachusetts private school I attended for six years, starting in the seventh grade.
>> Summary:
<pad>'resonance' is the literary magazine put out by the students of falmouth academy.</s>
------------
>> Original text:
During my time at F.A., I had at least one poem published in each issue of Resonance.
>> Summary:
<pad> i had at least one poem published in each issue of resonance.</s>
------------
>> Original text:
In high school, I was also a member of the staff.
>> Summary:
<pad> i was also a member of the staff in high school.</s>
------------
>> Original text:
But that wasn’t why I loved it.
>> Summary:
<pad> i loved the humour and the wit of the
------------
>> Original text:
I loved it — and I swear I am not exaggerating here — because I thought the writing in its pages was more beautiful than anything I’d ever read.
>> Summary:
<pad> i thought the writing in its pages was more

https://github.com/IlyaGusev/summarus

Мне кажется данных маловато, или эпох обучения. Я тот же mT5 дообучивал по рецептам от Ильи - всё офигенно суммаризует.

NoBadWordsLogitsProcessor посмотреть и переписать под свои нужды)

Как я понимаю, там нужно -inf скор выставить тому, что не в списке разрешённых

Что касается замены имён, можно попробовать генерировать несколько саммари, выделять все именованные сущности NER'ом, и понижать приоритет тех саммари, в которых появились сущности, не упомянутые в исходном тексте.